Reading original metadata.csv

In [4]:
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
module_path = os.getenv('MODULE_PATH') 
csv_path = os.path.join(module_path,"metadata.csv")
#select dataset from metadata
df=pd.read_csv(csv_path)
# df = df[(df["mode"] == 0) & (df["difficultyrating"]>4.8) & (df["difficultyrating"]< 6) & (df["split"]=="train") & (df['total_length'] < 600)]
# # df = df[df["audio"]=="0013ddfd8bd55fdccc0b253e313b7a60"]
# df = df[0:2]
df=df[df["bpm"]<100]
# df=df[df['BeatDivisor'] > 4]
df[["bpm","total_length"]]

,bpm,total_length
3,81.65,380
14,88.00,218
22,90.10,121
27,99.00,98
50,53.00,49
...,...,...
107898,97.00,207
107902,90.00,153
107933,80.00,86
107944,84.10,82


# Printing processed beatmap and audio from npy stored in processed-beatmaps

In [1]:
from Melokai.dataReadFromDisk import *
df = load_df_from_csv()

DataFrame loaded from: E:\Notebooks\osu\processed-metadata\processed-metadata.csv


In [2]:
beatmaps = []
audios = []
for index, row in df.iterrows():
    beatmap_path = os.path.join(project_path, "processed-beatmaps", f"{row['audio']}-b.npy")
    audio_path = os.path.join(project_path, "processed-audio", f"{row['audio']}-a.npy")
    beatmap_data = np.load(beatmap_path, allow_pickle=True)
    audio_data = np.load(audio_path, allow_pickle=True)
    beatmaps.append(beatmap_data)
    audios.append(audio_data)

# Beatmap parsin format

[
<br>(0) x-cordinate, 
<br>(1) y-cordinate,
<br>(2) timestamp(starting),
<br>(3) noteType(1,5 is circle ; 2,6 is slider; 8,12 is spinner),
<br>(4) hitSound: Type of hit sound (0 = normal, 1 = soft, 2 = drum),
<br>(5) If slider then SliderType : Assign B,L,P,O etc numbers 1,2,3,4,etc,
<br>(6) Same as X-cordinates i guess for now need to check,
<br>(7) Same as Y-cordinates i guess for now need to check,
<br>(8) Number of times whole slider collectively repeats itself (from start to end),
<br>(9) slider length of this slider part,
<br>(10) Edge Settings and Data for this node,
<br>(11) first part of Node Data (a:b) => a,
<br>(12) second part of Node Data (a:b) => b,
<br>(13) 1st Additional Data (a:b:c:d) => a,
<br>(14) 2nd Additional Data (a:b:c:d) => b,
<br>(15) 3rd Additional Data (a:b:c:d) => c,
<br>(16) 4th Additional Data (a:b:c:d) => d,
<br>(17) if slider then slider START(0), MIDDLE(1), END(2),
<br>(18) if slider/spinner then this slider node end TIMESTAMP
<br>(19) to (26) inherited timing points data
<br>(27) to (34) uninherited timing points data
<br>
]

In [5]:
for b in beatmaps[1]:
    print(b)

[  256.             192.           20181.              12.
     0.               0.               0.               0.
     0.               0.               0.               0.
     0.               0.               0.               0.
     0.               0.           25155.           20181.
   100.               0.               0.               0.
     0.               0.               0.             -29.
   315.7894736842     4.               2.               1.
    40.               1.               0.          ]
[  131.             192.           25234.               6.
     0.               1.             131.             192.
     1.             131.25             2.               0.
     2.               0.               0.               0.
     0.               0.           25510.           25234.
   100.               4.               2.               1.
    80.               0.               0.             -29.
   315.7894736842     4.               2.               1.
   